In [10]:
import os
import pandas as pd
import numpy as np
import cv2

In [8]:
# def json2df():
def cntarea(cnt):
    cnt = np.array(cnt)
    area = cv2.contourArea(cnt)
    return area

def cntperi(cnt):
    cnt = np.array(cnt)
    perimeter = cv2.arcLength(cnt,True)
    return perimeter

def cntMA(cnt):
    cnt = np.array(cnt)
    #Orientation, Aspect_ratio
    (x,y),(MA,ma),orientation = cv2.fitEllipse(cnt)
    return np.max((MA,ma)),np.min((MA,ma)),orientation

In [2]:
#input folder
jsonsrc = r'\\10.99.68.178\andreex\data\monkey fetus\gestational 40\hovernet_out\json'
#output folder creation
dst = os.path.join(os.path.dirname(jsonsrc), 'df')
if not os.path.exists(dst): os.mkdir(dst)

In [3]:
jsonnms = [i for i in os.listdir(jsonsrc) if i.endswith('json')]
for jsonnm in jsonnms:
    jsonpth = os.path.join(jsonsrc,jsonnm)
    imID,ext = os.path.splitext(os.path.basename(jsonpth))
    dstfn = os.path.join(dst, '{}.pkl'.format(imID))
    if os.path.exists(dstfn):
        print("pkl already exists, skipping the file ID {}".format(imID))
    try:
        json = pd.read_json(jsonpth, orient='index')
    except:
        print('cannot open json')
    json = pd.DataFrame(json[0].loc['nuc']).T.drop(columns=['type_prob'])
    json = json[json['contour'].map(len) > 5].reset_index(drop=True)
    json['Area'] = json['contour'].apply(lambda row: cntarea(row))
    json['Perimeter'] = json['contour'].apply(lambda row: cntperi(row))
    json['Circularity'] = 4 * np.pi * json['Area'] / json['Perimeter'] ** 2
    json['MA'] = json['contour'].apply(lambda row: cntMA(row))
    json[['MA', 'ma', 'orientation']] = pd.DataFrame(json.MA.tolist())
    json['AspectRatio'] = json['MA'] / json['ma']
    json['imID'] = imID
    json.to_pickle(dstfn)

'monkey_fetus_40_0301.json'